In [1]:
import os
import gc
import math
import json
from datetime import datetime
import shutil
from pathlib import Path
import optuna
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from sklearn.model_selection import GroupKFold
#from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import DataLoader
from transformers import BatchEncoding, BertTokenizerFast
from typing import Dict, List, Tuple, NamedTuple
import scml
import mylib

In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    stride: int
    batch_size: int
    gradient_checkpointing: bool


class Conf(NamedTuple):
    sample_frac: float = 1
    epochs: int = 30
    lr: Tuple[float, float] = (1e-3, 1e-3)
    multi_sample_dropout_size: Tuple[int, int] = (8, 8)
    multi_sample_dropout_increment: Tuple[float, float] = (0, 0)
    swa_start_epoch: Tuple[int, int] = (-1, -1)
    swa_anneal_epochs: Tuple[int, int] = (3, 3)
    pretrained_dir: Path = Path("pretrained")
    hidden_size: int = 32
    model_max_length: int = 32
    stride: int = 0
    batch_size: int = 4
    model_name: str = "roberta_to_roberta"
    gradient_checkpointing: bool = False
    gpus: List[int] = [0]
    patience: int = 1
    n_trials: int = 1
    n_folds: int = 3
    seed: int = 31
    
        
conf = Conf()
print(conf)

Conf(sample_frac=1, epochs=30, lr=(0.001, 0.001), multi_sample_dropout_size=(8, 8), multi_sample_dropout_increment=(0, 0), swa_start_epoch=(-1, -1), swa_anneal_epochs=(3, 3), pretrained_dir=WindowsPath('pretrained'), hidden_size=32, model_max_length=32, stride=0, batch_size=4, model_name='roberta_to_roberta', gradient_checkpointing=False, gpus=[0], patience=1, n_trials=1, n_folds=3, seed=31)


In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(conf.seed)
pl.seed_everything(conf.seed)

Global seed set to 31


31

In [4]:
device: torch.device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce GTX 1060 6GB
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("tokenizer")
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='tokenizer', vocab_size=1855612, model_max_len=32, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<s>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['<click>', '<cart>', '<order>']})


In [ ]:
%%time
with open("input/x.json") as f:
    x = json.load(f)
with open("input/y.json") as f:
    y = json.load(f)
with open("input/sids.json") as f:
    sids = json.load(f)
print(f"len(x)={len(x['input_ids'])}\nlen(y)={len(y['input_ids'])}\nlen(sids)={len(sids)}")
assert len(sids)==len(y["input_ids"])
assert len(sids)==len(x["input_ids"])

In [ ]:
ds = mylib.OttoDataset(
    BatchEncoding(x), 
    labels=y["input_ids"],
    session_ids=sids,
)
shape = (len(ds), ds.seqlen())
print(f"ds.shape={shape}\n{ds[30]}\n{ds[0]}")

In [ ]:
job_ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = Path("models") / conf.model_name / job_ts
job_dir.mkdir(parents=True, exist_ok=True)
print(f"job_dir={job_dir}")

# Train final model on best Hps

In [ ]:
%%time
splitter = GroupKFold(n_splits=50)
dummy = np.zeros(len(ds))
for ti, vi in splitter.split(dummy, y=ds.stratification(), groups=ds.session_ids):
    tra_ds = torch.utils.data.Subset(ds, ti)
    val_ds = torch.utils.data.Subset(ds, vi)
    break
print(f"len(tra_ds)={len(tra_ds)}, len(val_ds)={len(val_ds)}")

In [ ]:
#best = df.iloc[0].to_dict()
best = {
    "lr": conf.lr[0],
}
print(f"best={best}")

In [ ]:
model = mylib.OttoLightningModel(
    lr=best["lr"],
    decoder_start_token_id=tokenizer.cls_token_id,
    pad_token_id=tokenizer.pad_token_id,
    vocab_size=len(tokenizer),
    hidden_size=conf.hidden_size,
)
print(model.model.config)

In [ ]:
print(model.model)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
trainer = mylib.Trainer(
    default_root_dir=job_dir,
    gpus=conf.gpus,
    max_epochs=conf.epochs,  
    callbacks=mylib.training_callbacks(patience=conf.patience),
    deterministic=False,
)
trainer.fit(
    model,
    train_dataloaders=DataLoader(
        tra_ds,
        batch_size=conf.batch_size,
        shuffle=True,
        num_workers=0,
    ),
    val_dataloaders=DataLoader(
        val_ds,
        batch_size=conf.batch_size,
        shuffle=False,
        num_workers=0,
    ),
)

In [ ]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")
#print(f"Saved {str(job_dir)}")